<a href="https://colab.research.google.com/github/apapaioannou92/BA/blob/HW1/aap2204_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IEOR 4650  Business Analytics (Fall 2019)**

**Homework 1 [Total points: 60]**

** Due: 11:59 PM, September 28**

This first homework consists of Python programming practices and the applications to data clearning and linear regression. 

The steps to finish in the homework:

*   Step 1: Make a copy of the file to your LionDrive.
*   Step 2: Work with the copy (an ipynb file).
*    <font color='red'>**Step 3: Rename the copy to your_uni_hw1.ipynb (For example: yd4501_hw1.ipynb).**</font>  
*   Step 4: Submit the copy on CourseWorks.

Before you submit your assignment, make sure to re-run your code from the beginning. (You can do so by first clicking Runtime/Reset All Runtimes and then clicking Runtime/Run all)


**All the questions in this homework requires coding**

In [0]:
#This imports all the packages you need for the homework
#Please run this first


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#import modules

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt



## Exercise 1 [Total: 22 Points]

In this exercise, we will work on some exercises that practices our data manipulation skills.




<div>
<img src="https://http2.mlstatic.com/poster-pokemon-full-hd-42x29cm-frete-gratis-D_NQ_NP_752705-MLB25053489227_092016-F.jpg" width="250"/>
</div>

## Setting
Pokemon is a very popular video game, in which the trainer catches pokemon and use them for battling. Each pokemon has its name and its pokedex. A Pokemon belongs to at least 1 and at most 2 Pokemon types.  There are different stats to measure the combat power of a pokemon, which include HP, Attack, Defense, Special Attack Strength, Special Defensive Strength, and Speed. Pokemon are usually released in batches. Thus, they have different generations. In addition, a pokemon can either be a regular pokemon and legendary pokemon. 



|Variable| Description|
|---|---|
|PokeDex |PokeDex index number|
| Name | Name of the Pokemon  |  
| Type 1 | Type of pokemon  |  
| Type 2 | Other Type of Pokemon  |  
| HP |  Hit Points  |  
| Attack |  Attack Strength  |  
| Defense |  Defensive Strength  |  
| Sp. Atk |  Special Attack Strength  |  
| Sp. Def |  Special Defensive Strength  |  
| Speed | Speed  |  
| Generation | Number of generation  |  
| Legendary |  True if Legendary Pokemon False if not | "

In [85]:
#Load the dataset 
link="https://drive.google.com/open?id=1wX36WLNOPLPVU7BCA-J9bBhGZPwuFqol"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile.csv')  
Pokemon = pd.read_csv('myfile.csv')
Pokemon.head(5)

,PokeDex,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Charmander,Fire,***,39,52,43,60,50,65,1,False
4,5,Charmeleon,Fire,***,58,64,58,80,65,80,1,False


### Q1 [2 points]
* Find the number of unique pokemon in the table. [hint: you want to make sure the same pokemon name does not appear twice in the table]

**Make sure to print out the result**

In [86]:
Pokemon["Name"].nunique

<bound method IndexOpsMixin.nunique of 0                  Bulbasaur
1                    Ivysaur
2                   Venusaur
3                 Charmander
4                 Charmeleon
5                  Charizard
6                   Squirtle
7                  Wartortle
8                  Blastoise
9                   Caterpie
10                   Metapod
11                Butterfree
12                    Weedle
13                    Kakuna
14                  Beedrill
15                    Pidgey
16                 Pidgeotto
17                   Pidgeot
18                   Rattata
19                  Raticate
20                   Spearow
21                    Fearow
22                     Ekans
23                     Arbok
24                   Pikachu
25                    Raichu
26                 Sandshrew
27                 Sandslash
28                  Nidoran♀
29                  Nidorina
               ...          
691                Clauncher
692                Clawitzer
693 

### Q2. [4 points]

* Add one additional column named "Total" to the table 
> The value of Total for each pokemon is equal to the sum of all the stats numbers for each pokemon (HP+Attack+Defense+Sp. Atk+Sp. Def+ Speed) and is the measure of total combat power of a pokemon.
* drop the stats that are used to compute "Total" from the table

**Make sure to print out the result for the last step**

In [87]:
Pokemon = Pokemon.assign(Total=Pokemon['HP']+Pokemon['Attack']+Pokemon['Defense']+Pokemon['Sp_Atk']+Pokemon['Sp_Def']+Pokemon['Speed'])
#OR 
# Pokemon['Total'] = Pokemon['HP']+Pokemon['Attack']+Pokemon['Defense']+Pokemon['Sp_Atk']+Pokemon['Sp_Def']+Pokemon['Speed']
Pokemon.drop(['HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed'], axis=1, inplace=True)
Pokemon.head()

,PokeDex,Name,Type_1,Type_2,Generation,Legendary,Total
0,1,Bulbasaur,Grass,Poison,1,False,318
1,2,Ivysaur,Grass,Poison,1,False,405
2,3,Venusaur,Grass,Poison,1,False,525
3,4,Charmander,Fire,***,1,False,309
4,5,Charmeleon,Fire,***,1,False,405


### Q3 [4 points]

* Create a sub-table "Pokemon_sub" that includes only the pokemon name and the Pokedex for all the **Legendary**  pokemon from **Generation 1** and **Generation 2**. 

**Make sure to print out the content of the sub-table**

In [88]:
Pokemon_sub=Pokemon[(Pokemon["Legendary"]==True) & (Pokemon["Generation"]==1) | (Pokemon["Generation"]==2)]
Pokemon_sub.head()
# OR
# filter_ = (Pokemon["Generation"]==1) | (Pokemon["Generation"]==2) & (Pokemon['Legendary']=="True")
# Pokemon_sub = Pokemon.loc[filter_, ['Name', 'PokeDex']]
# Pokemon_sub.head()

,PokeDex,Name,Type_1,Type_2,Generation,Legendary,Total
143,144,Articuno,Ice,Flying,1,True,580
144,145,Zapdos,Electric,Flying,1,True,580
145,146,Moltres,Fire,Flying,1,True,580
149,150,Mewtwo,Psychic,***,1,True,680
151,152,Chikorita,Grass,***,2,False,318


###Q4 [5 points]

A pokemon might belong to only one type or two types. If a pokemons belongs to only one type, "Type_2" field will be missing that Pokemon

We call a pokemon that belongs to only one type a "single-type" pokemon and a pokemon that belongs to two types a "double-type" pokemon. For example, Snorlax is a "single-type" pokemon and belongs to Normal type. For another example, Bulbasaur is a "double-type" pokemon and belongs to both Poison type and Grass type. 	  

* Convert the missing values to a format that can be recognized by Pandas.
* Report for each pokemon type, how many "single-type" pokemon exists.

**Make sure to print out the result for the last step**

In [89]:
# import numpy as np # unnecessary because it is already imported in the preamable
Pokemon['Type_2']=Pokemon['Type_2'].replace("***",np.nan)

Pokemon_single_type=Pokemon[(Pokemon["Type_2"].isna()==True)]
Pokemon_single_type["Type_1"].value_counts()

# OR equivalently
# Pokemon_single_type = Pokemon[pd.isnull(Pokemon['Type_2'])]
# Pokemon_single_type["Type_1"].value_counts()

Normal      60
Water       57
Grass       33
Psychic     32
Fire        28
Electric    26
Fighting    20
Bug         17
Poison      15
Fairy       15
Ground      13
Ice         12
Dragon      11
Ghost        9
Rock         9
Dark         9
Steel        4
Flying       1
Name: Type_1, dtype: int64

###Q5 [7 points]

It is important to have the ability of learning programming by yourself. Let's use some adequate methods to find the pokemon with the highest total combat power in each type.  

In this table, you should have 3 columns

(1) column 1: Type name

(2) column 2: name of the most power pokemon for this pokemon type

(3) column 3: total combat power for this pokemon

[hint: You might find the following four methods to be relevant. However, there are multiple ways to achieve the goal. You do not need to use them in your answer. 

> `concat` method

> `sort_values` method

> `groupby` method

> `first` method

In [90]:
Pokemon_Type_1_strongest=Pokemon.groupby('Type_1',as_index=False).max()
Pokemon_Type_1_strongest=Pokemon_Type_1_strongest.drop(columns=['PokeDex','Generation','Legendary'])
Pokemon_Type_1_strongest=Pokemon_Type_1_strongest.rename(columns={'Type_1': 'Type'})
Pokemon_Type_1_strongest.head()

Pokemon_Type_2_strongest=Pokemon.groupby('Type_2',as_index=False).max()
Pokemon_Type_2_strongest=Pokemon_Type_2_strongest.drop(columns=['PokeDex','Generation','Legendary','Type_1'])
Pokemon_Type_2_strongest=Pokemon_Type_2_strongest.rename(columns={'Type_2': 'Type'})

Pokemon_final_strongest=pd.concat([Pokemon_Type_1_strongest,Pokemon_Type_2_strongest])
Pokemon_final_strongest=Pokemon_final_strongest.sort_values("Total", ascending=False).groupby('Type',as_index=False).first()
Pokemon_final_strongest.head()

,Type,Name,Total
0,Bug,Yanmega,600
1,Dark,Zweilous,680
2,Dragon,Zygarde50% Forme,680
3,Electric,Zekrom,680
4,Fairy,Xerneas,680


## Exercise 2 [Total: 38 points]



###Setting
In this exercise, we will work with a dataset which explores the store sales.

Before doing the assignment, please study this dataset here https://www.kaggle.com/c/rossmann-store-sales/data


In [91]:
#Load the dataset
link="https://drive.google.com/open?id=1Q6J0q4tlWJ7TajX6hMSvZ8tK3CyDD8QY"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Sales.csv')  
Sales = pd.read_csv('Sales.csv')
Sales.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


*You might see a warning when importing the data. This is because StateHoliday has both values in string and numbers. No worries this for now.*

### Q1 [7 points]

Run the following model

$$Sales=\beta_0+\beta_1 Customers + \epsilon$$

> (1) Print out the summary output

> (2) Based on regression result, answer the following questions

>> Interpret  $\hat{\beta_0}$ and $\hat{\beta_1}$ based on the regression output. 

>> Reason whether there is a negative correlation between the number of customers and Sales

>> Report the 99% confidence interval for $\beta_1$

In [92]:
from statsmodels.formula.api import ols
#running the model 𝑆𝑎𝑙𝑒𝑠=𝛽0+𝛽1𝐶𝑢𝑠𝑡𝑜𝑚𝑒𝑟𝑠+𝜖 
model = ols(formula="Sales~Customers",data=Sales).fit()
#print the summary output of the model, and the confidence interval
print(model.summary(), model.conf_int(0.01), sep='\n \n \n')

# The interpretations of 2(a) and (b), as well as the 99% confidence interval for  𝛽1,  follow after the summary output.

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                 4.082e+06
Date:                Sun, 29 Sep 2019   Prob (F-statistic):               0.00
Time:                        01:36:29   Log-Likelihood:            -9.0214e+06
No. Observations:             1017209   AIC:                         1.804e+07
Df Residuals:                 1017207   BIC:                         1.804e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1077.7365      2.883    373.869      0.0

### $\beta_{1}$ interpretation
If you increase the Customers independent variable by 1 unit without changing other factors, the expected value for the Sales target variable will increase ON AVERAGE by 7.4171 (+- 0.004)

### $\beta_{0}$ interpretation
If the Customers independent variable is equal to 0, the expected value for the Sales target variable will be on average equal to 1077.7365 (+- 2.883)

Also:
The intercept (often labeled the constant-here $\beta_{0}$) is the expected mean value of Sales when all independent variables, in our case Customers=0. 

If X (array of independent variables-in our case just Customers) sometimes equals 0, the intercept is simply the expected mean value of Y (target variable-in our case, Sales) at that value. If X never equals 0, then the intercept has no intrinsic meaning.

### Reason whether there is a negative correlation between the number of customers and Sales
From the regression table we observe that there is not a negative correlation (instead there is a positive correlation) between the Sales (target/dependent) variable and the Customers (predictor/independent) variable. The reason why the independent variable, Customers, does not have a negative correlation with the Sales target variable is that, as the number of customers increases, the dollar amount of Sales obtained is expected to be higher, given that higher exposure leads to higher returns and no other restrictions take place.

### 99% confidence interval for $\beta_{1}$
The 99% confidence interval of $\beta_{1}$ is the interval [7.407605, 7.426518].

###Q2. [7 points]

* For the model above, use a Python attribute we discussed in class to retrieve the values for $\hat{\beta_0}$ and $\hat{\beta_1}$.

* Based on these two values, give the prediction for $\widehat{Sales}$ when number of customers=110.

* Find out the average of the actual Sales price for all the records that have customers=110.

In [93]:
#estimated values of beta
# model.params

# For the model above, use a Python attribute we discussed in class to print the values for  𝛽0^  and  𝛽1^ .

# Assign the intercept, or 𝛽0^, to a variable
b0_coeff = model.params[0]
intercept = b0_coeff

# Assign the first coefficient, 𝛽1^, to a variable
b1_coeff = model.params[1]
b1_coeff

# Print the intercept, or 𝛽0^, and the first coefficient, 𝛽1^.
print(b0_coeff, b1_coeff)


# Based on these two values of the beta coefficients, print the predicted value for Sales when the number of Customers = 110
Sales_110 = b0_coeff + b1_coeff * 110
print(Sales_110)


# Find out the average of the actual Sales price for all the records that have customers=110.
print(Sales['Sales'].loc[Sales['Customers'] == 110].mean())

1077.7365031848192 7.417061572866315
1893.613276200114
908.1428571428571


###Q3 [8 points]

One person argues the following:

* He found from the data that we should expect 0 Sales when we have 0 customers
* Since $\beta_0$ is the expected Sales price, when the number of customers is equal to 0, it should be set equals to 0.  

> Based on data, support that his first argument is true.

> Provide an argument on why the second point might be problematic. 

You will find the following articles useful:
[Article 1](https://dynamicecology.wordpress.com/2017/04/13/dont-force-your-regression-through-zero-just-because-you-know-the-true-intercept-has-to-be-zero/)

In [94]:
# His first argument makes sense and it can be verified by calculating and printing the following quantity, ie actual (sample's) average of Sales when the customers are equal to 0
print(Sales['Sales'].loc[Sales['Customers'] == 0].mean())

# Below, we provide an argument and the reasoning why his second point might be problematic.

0.0


His second argument is wrong. More specifically, the linear form of the function of our regression model(s) is ordinarily chosen for convenience and/or because it fits the observed data reasonably well, or for other reasons. So the main reason why his second argument is wrong, it is because we do not know the actual form of F(X) outside of the origin. For that reason we cannot risk forcing the intercept of our regression line through the origin (0,0), and making it less accurate in all other X regions away from the origin.

Moreover, the regression line that results from our linear model may provide a reasonable fit overall throughout the range of X values observed in our dataset,but with a nonsensical estimated intercept. This is the way that our linear regression model is telling us that the true shape of F(X) is following another model and it is nonlinear near and at the origin. By forcing the intercept through the origin could give us a more accurate prediction for our target variable at the origin, but:

1) it would fail to capture the true and actual behavior of the Sales target variable near and at the origin, and

2) it would impact the accuracy of the predicted values of our target variable in all other areas away from the origin.

###Q4. [5 points] 

Now, run the following model:

$Sales=\beta_0+\beta_1 Customers +\beta_2 Open + \epsilon$

> Fit the model and print out the summary table.

> Interpret $\hat{\beta_0}$ , $\hat{\beta_1}$ and $\hat{\beta_2}$. 


In [95]:
model = ols(formula="Sales~Customers+Open",data=Sales).fit()
# Print results of the model and summary table
print(model.summary())

# Interpreting 𝛽0^, 𝛽1^, and 𝛽2^

# The intercept is negative. More specifically, for 0 customers and for Open = 0, which means for stores that are closed, the expected value for the target variable named Sales 
# is on average -2.7e-11 ± 3.859, which is practically 0.

# There is a positive correlation between the independent/predictor variable named Customers and the dependent/target variable named Sales. More specifically, for 1 unit increase
# of the independent/predictor variable named Customers, while maintaining all the other independent variables constant, the expected value for the dependent/target variable named 
# Sales is expected to increase on average by 6.3720 ± 0.004 units.

# There is a positive correlation between the independent/predictor variable named Open and the dependent/target variable named Sales. More specifically, for 1 unit increase 
# of the independent/predictor variable named Open (so from closed stores to open stores), while maintaining all the other independent variables constant, the expected value for 
# the dependent/target variable named Sales is expected to increase on average by 2095.4121 ± 5.380 units.

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.826
Method:                 Least Squares   F-statistic:                 2.421e+06
Date:                Sun, 29 Sep 2019   Prob (F-statistic):               0.00
Time:                        01:36:30   Log-Likelihood:            -8.9507e+06
No. Observations:             1017209   AIC:                         1.790e+07
Df Residuals:                 1017206   BIC:                         1.790e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -2.7e-11      3.859     -7e-12      1.0

###Q5. [4 points]

Based on regression, prove that Q4 is equivalent to regressing "Sales" on "Customers" using only observations with stores that are open. 

In [96]:
# We keep only the dataset's obsrevations with stores that are open:
filter_ = (Sales["Open"] == 1)
Sales = Sales[filter_]
# Sales["Open"].nunique()
# Sales["Open"].value_counts()
model = ols(formula="Sales~Customers",data=Sales).fit()
print(model.summary(), sep='\n \n \n')
print('\n')

# We observe that the beta coefficient, β1^, in both models is exactly the same and there is very slight difference only in its standard deviation value. 

# Moreover, the intercept, beta coefficient, β0^, is shifted upwards from -2.7e-11 (practically ~0) to 2095.4121, an increase which is exactly equal to the coefficient of the 
# independent/predictor variable named Open of the model of Q4, which makes sense, since the independent/predictor variable named Open in the model of Q4 could take the values of 
# 0 or 1, and in our current model of Q5 (open stores) if the independent/predictor variable named Open existed, it should take the value 1 for all the observations of our model.
# That is why the intercept is shifted upwards by 2095.4121 in the model of Q5; it is shifted upwards in order for all the predicted values of our model to integrate the increase 
# of 2095.4121 * 1, which is equivalent to let the variable Open=1 (open stores) in the model of Q4. 

# Thus the model of Q4 is equivalent to regressing "Sales" on "Customers" using only observations with stores that are open (=model of Q5).

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                 1.780e+06
Date:                Sun, 29 Sep 2019   Prob (F-statistic):               0.00
Time:                        01:36:30   Log-Likelihood:            -7.5086e+06
No. Observations:              844392   AIC:                         1.502e+07
Df Residuals:                  844390   BIC:                         1.502e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2095.4121      4.116    509.149      0.0

###Q6. [7 points]
Store table gives the information of each store. 

* Merge Sales table with Store table
* Run the following regression model

$$Sales=\beta_0+\beta_1 Customers +  \beta_2 Open + \beta_3 Competition Distance+\epsilon$$

* Based on the regression result, interpret $\widehat{\beta_3}$. Is this consistent with your expectation? Why?

In [100]:
link="https://drive.google.com/open?id=1g6URzUJnhCLOtg0a9TTHC18KYcqwxvoc"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile_store.csv')  
Store = pd.read_csv('myfile_store.csv')
# Store.head(5)

# Reloading the initial Sales dataset into a df

Sales = pd.read_csv('Sales.csv')
# Sales.head()

# Merging the Sales and Store dfs

Sales_Store_merged = Sales.merge(Store, on = "Store") # since Store is the only 1 common column in both dfs
Sales_Store_merged.head()

# Running the model as described in the question and printing the model summary

model = ols(formula="Sales~Customers+Open+CompetitionDistance",data=Sales_Store_merged).fit()
print(model.summary(), sep='\n \n \n')
# print(Sales.shape[0],Store.shape[0])

# Interpreting 𝛽3ˆ based on the regression result, and if this is consistent with our expectation and why

# The value of coefficient 𝛽3ˆ of the independent/predictor variable named CompetitionDistance is positive but very small, ie 0.0296. The sign of the coefficient b3 of the 
# independent/predictor variable named CompetitionDistance makes sense, since we were expecting that as the competition distance increases, the dependent/target variable named
# Sales is expected to increase, since no competition is present. The value of the b3 coefficient can be interpreted as follows: 1 unit increase in the value of the 
# independent/predictor variable named CompetitionDistance is expected to lead in an average increase of 0.0296 ± 0.000 units in the expected value of the dependent/target 
# variable named Sales. It worths of mention, the fact that the value of the coefficient 𝛽3ˆ is small, since the data points/observations of the Store dataset are very few compared 
# to the data points/observations (rows) of the Sales dataset, thus the value of the coefficient 𝛽3ˆ is not accurate (because of the existence of a lot of NaNs).

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                 1.647e+06
Date:                Sun, 29 Sep 2019   Prob (F-statistic):               0.00
Time:                        02:27:16   Log-Likelihood:            -8.9181e+06
No. Observations:             1014567   AIC:                         1.784e+07
Df Residuals:                 1014563   BIC:                         1.784e+07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -156.6383    

In [0]:
### PLEASE IGNORE THIS
#@title
# Also, based on the fact that the order of magnitude of the value of the b3 coefficient seems small, compared to those of the other beta coefficients, someone could falsely suggest 
# that the small order of magnitute indicates that the independent/predictor variable named CompetitionDistance does not have a big impact in the prediction of the Sales. 
# The small order of magnitude in the the coefficient b3 though, could probably be attributed to the fact that the units of CompetitionDistance in our model are a few orders of 
# magnitude higher in comparison to those of the other independent/predictor variables. In order to account for that, the proper thing to do would be to normalize all the features 
# of our model, and then run the regression model, so that the impact of the scales of the features do not lead us to misinterpretations (would be a result of misleading values of 
# the beta coefficients of our model), regarding the actual impact of the independent (predictor) variables to the dependent (target) variable.